In [1]:
import syft as sy
import numpy as np
import pandas as pd
from syft.core.adp.entity_list import EntityList
from time import time, sleep

In [2]:
t0 = time()
# df = pd.read_parquet("/home/ruchi/1B_rows_dataset_sample.parquet")
df = pd.read_parquet("1M_rows_dataset_sample.parquet")
tf = time() - t0

print(f"Time taken to read file with {df.shape[0]/1e6} million rows : {tf} seconds")

Time taken to read file with 1.0 million rows : 0.20989513397216797 seconds


In [3]:
df['impressions'].nunique()

108

In [4]:
df['user_id'].nunique()

173730

In [5]:
df.head()

,tweet_id,impressions,tweet_date_time,date,time,user_id,url,publication_title,ad_fontes_bias,ad_fontes_reliability,domain
0,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,92256,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
1,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,418264,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
2,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,195053,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
3,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,590420,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com
4,1,9029,2021-03-21 06:29:10,2021-03-21,06:29:10,602475,https://www.aljazeera.com/news/2019/04/trump-v...,Al Jazeera,-0.75,48.0,.aljazeera.com


In [6]:
scale = 1 #10_000  # This is put here to stop int32 overflow errors

t0 = time()
impressions = df['impressions'].to_numpy(np.int64) // scale
data_subjects = EntityList.from_series(df['user_id'])
tf = time() - t0

print(f"Time taken to create inputs for Syft Tensor: {tf} seconds")

Time taken to create inputs for Syft Tensor: 0.023783445358276367 seconds


In [7]:
df['impressions'].min()

726

In [8]:
f = lambda sigma: np.square(726)/(2*(sigma**2))

In [9]:
f(1e5)

2.63538e-05

In [10]:
t0 = time()
# tweets_data = sy.Tensor(impressions).private(min_val=750_000/scale, max_val=20e6/scale, entities = data_subjects,ndept=True)  # RUn this for 1 billion rows
tweets_data = sy.Tensor(impressions).private(min_val=500/scale, max_val=25000/scale, entities = data_subjects,ndept=True)  # Run this for 1 million rows
tf = time() - t0

print(f"Time make Private Syft Tensor: {tf} seconds")

Time make Private Syft Tensor: 0.002109050750732422 seconds


In [11]:
domain_node = sy.login(email="info@openmined.org",password="changethis",port=8081)


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to None... done! 	 Logging into kind_goodfellow... done!


In [12]:
%%time
name = f"Tweets data- {time()}"

domain_node.load_dataset(
    assets={name: tweets_data},
    name=name,
    description=" Tweets- 1B rows",
    use_blob_storage=True
)

kj/filesystem-disk-unix.c++:1690: warning: PWD environment variable doesn't match current directory; pwd = /home/ruchi/PySyft


Loading dataset... checking asset types...                                                                                                                                    

Uploading Tweets data- 1648063626.8318357: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 37.45it/s]

Loading dataset... uploading...                        

Loading dataset... uploading... SUCCESS!                        

Run <your client variable>.datasets to see your new dataset loaded into your machine!
CPU times: user 58.6 ms, sys: 19.6 ms, total: 78.2 ms
Wall time: 277 ms


In [13]:
domain_node.datasets

Idx,Name,Description,Assets,Id
[0],Tweets data- 1648062991.2597036,Tweets- 1B rows,"[""Tweets data- 1648062991.2597036""] ->",69a7c9fe-f157-483a-8110-236823e27465
[1],Tweets data- 1648063276.7922251,Tweets- 1B rows,"[""Tweets data- 1648063276.7922251""] ->",ffba7595-942e-45cd-b619-14e1a1a8842e
[2],Tweets data- 1648063477.1145127,Tweets- 1B rows,"[""Tweets data- 1648063477.1145127""] ->",03398eff-a42c-4781-b22e-779ba084bd79
[3],Tweets data- 1648063626.8318357,Tweets- 1B rows,"[""Tweets data- 1648063626.8318357""] ->",3e7ac0ab-0a9c-4ca0-acd2-ee57df7c01dc


In [14]:
data = domain_node.datasets[-1][name]

In [15]:
%%time
sum_result = data.sum()
sum_result.block

CPU times: user 10.3 ms, sys: 3.6 ms, total: 13.9 ms
Wall time: 401 ms


<TensorPointer -> kind_goodfellow:2a0f45e93a654f71b2b60fcaa6b30553>

In [16]:
sum_result.exists

True

In [17]:
published_result = sum_result.publish(sigma=1e5)

Sigma, PB_spend
- 100, 859.4086891067999
- 1e4, 7.766216625311721
- 1e5, 7.766216625311721
- 2e5, 7.766216625311721
- 5e5, 7.766216625311721
- 1e7, 7.766216625311721
- 1e9, 7.766216625311721

Is PB not being calculated correctly?

In [18]:
while not published_result.exists:
    sleep(1)

In [19]:
published_result

<FloatPointer -> kind_goodfellow:38aeab5bb76f4a55868ca4208381f8a1>

In [20]:
published_result.request()

In [21]:
domain_node.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Jane Doe,info@openmined.org,Owner,DATA,accepted,,<UID: 0738402d87d748b99b6a3a0acebb2087>,<UID: 9438bf9f77ab49d684a9b87a1d7f431a>,[],None,None
1,Jane Doe,info@openmined.org,Owner,DATA,accepted,,<UID: a03df2207db44c00966618e2ea0cdf57>,<UID: 43734f27aada402da4f6a663b6b22389>,[],None,None
2,Jane Doe,info@openmined.org,Owner,DATA,pending,,<UID: 66d15214e19842fe9efca3a695e32205>,<UID: 38aeab5bb76f4a55868ca4208381f8a1>,[],None,None


In [22]:
domain_node.requests[-1].accept()

In [23]:
domain_node.requests

,Name,Email,Role,Request Type,Status,Reason,Request ID,Requested Object's ID,Requested Object's tags,Requested Budget,Current Budget
0,Jane Doe,info@openmined.org,Owner,DATA,accepted,,<UID: 0738402d87d748b99b6a3a0acebb2087>,<UID: 9438bf9f77ab49d684a9b87a1d7f431a>,[],None,None
1,Jane Doe,info@openmined.org,Owner,DATA,accepted,,<UID: a03df2207db44c00966618e2ea0cdf57>,<UID: 43734f27aada402da4f6a663b6b22389>,[],None,None
2,Jane Doe,info@openmined.org,Owner,DATA,accepted,,<UID: 66d15214e19842fe9efca3a695e32205>,<UID: 38aeab5bb76f4a55868ca4208381f8a1>,[],None,None


In [24]:
result_of_our_hard_labour = published_result.get()

In [25]:
result_of_our_hard_labour

array(-943004352, dtype=int32)

In [26]:
impressions.sum()

11941995258

In [27]:
def percentage_error(true_value, noisy_value):
    return (true_value - noisy_value)/true_value * 100

In [28]:
percentage_error(true_value=impressions.sum(), noisy_value=result_of_our_hard_labour)

107.89653932719725

In [29]:
(1644527104 - 11941995258)/11941995258 * 100

-86.22904239642598

In [30]:
11941995258 /1e6

11941.995258

In [31]:
domain_node.privacy_budget

9999999.0